In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/speaker-test/S_1_02.wav
/kaggle/input/speaker-test/S_1_03.wav
/kaggle/input/speaker-test/S_1_05.wav
/kaggle/input/speaker-test/S_1_04.wav
/kaggle/input/speaker-test/S_1_07.wav
/kaggle/input/speaker-test/S_1_08.wav
/kaggle/input/speaker-test/S_1_09.wav
/kaggle/input/speaker-test/S_1_01.wav
/kaggle/input/speaker-test/S_1_10.wav
/kaggle/input/speaker-test/S_1_06.wav
/kaggle/input/speaker-verification/true_test/S_10_06.wav
/kaggle/input/speaker-verification/true_test/S_10_02.wav
/kaggle/input/speaker-verification/true_test/S_10_10.wav
/kaggle/input/speaker-verification/true_test/S_10_09.wav
/kaggle/input/speaker-verification/true_test/S_10_05.wav
/kaggle/input/speaker-verification/true_test/S_10_08.wav
/kaggle/input/speaker-verification/true_test/S_10_01.wav
/kaggle/input/speaker-verification/true_test/S_10_07.wav
/kaggle/input/speaker-verification/true_test/S_10_04.wav
/kaggle/input/speaker-verification/true_test/S_10_03.wav
/kaggle/input/speaker-verification/false_test/Spea

In [2]:
import os
import random

import numpy as np
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, accuracy_score

# Dataset paths
GENUINE_DIR = '/kaggle/input/speaker-verification/true_speaker'
IMPOSTER_DIR = '/kaggle/input/speaker-verification/false_speaker'

# Audio & Model Configuration
SR = 16000       # Sampling Rate
N_MELS = 40      # Mel Filterbanks
DURATION = 3.0   # Audio Duration (seconds)
EMBED_DIM = 128  # Speaker Embedding Size

# Data Split Config
ENROLL_N = 5            # Genuine samples for enrollment
DEV_N = 60              # Genuine samples for threshold tuning
IMPOSTER_DEV_PER = 60   # Imposter samples for dev set

In [3]:
# Load genuine files (UNCHANGED)
genuine_files = sorted([
    os.path.join(GENUINE_DIR, f)
    for f in os.listdir(GENUINE_DIR)
    if f.lower().endswith('.wav')
])
print(f'Found {len(genuine_files)} genuine files')

# Load imposter speakers
imposter_speakers = [
    d for d in sorted(os.listdir(IMPOSTER_DIR))
    if os.path.isdir(os.path.join(IMPOSTER_DIR, d))
]

imposter_files = {}
for spk in imposter_speakers:
    spk_dir = os.path.join(IMPOSTER_DIR, spk)
    files = sorted([
        os.path.join(spk_dir, f)
        for f in os.listdir(spk_dir)
        if f.lower().endswith('.wav')
    ])
    imposter_files[spk] = files

print(f"Found {len(imposter_speakers)} imposter speakers")
print(f"Total imposter files: {sum(len(v) for v in imposter_files.values())}")

# Shuffle
random.seed(42)
random.shuffle(genuine_files)
for k in imposter_files:
    random.shuffle(imposter_files[k])

# Split genuine files
g_enroll = genuine_files[:ENROLL_N]
g_dev    = genuine_files[ENROLL_N:ENROLL_N+DEV_N]
g_test   = genuine_files[ENROLL_N+DEV_N:]

print(f"Enroll: {len(g_enroll)}, Dev: {len(g_dev)}, Test: {len(g_test)}")

# Split imposters dev/test
imp_dev = []
imp_test = []
for spk, files in imposter_files.items():
    imp_dev.extend(files[:IMPOSTER_DEV_PER])
    imp_test.extend(files[IMPOSTER_DEV_PER:])

print(f"Imposter dev: {len(imp_dev)}, Imposter test: {len(imp_test)}")

Found 90 genuine files
Found 20 imposter speakers
Total imposter files: 2004
Enroll: 5, Dev: 60, Test: 25
Imposter dev: 1200, Imposter test: 804


In [4]:
# SpeakerCNN
class SpeakerCNN(nn.Module):
    def __init__(self, num_speakers, embedding_dim=EMBED_DIM):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.pool = nn.MaxPool2d(2)
        self.adapool = nn.AdaptiveAvgPool2d((8, 8))

        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, embedding_dim)

        self.classifier = nn.Linear(embedding_dim, num_speakers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)

        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)

        x = self.adapool(x)
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        emb = self.fc2(x)
        emb = F.normalize(emb, p=2, dim=1)

        logits = self.classifier(emb)
        return emb, logits

In [5]:
# Log-Mel Feature Extraction
def extract_logmel(file_path, sr=SR, n_mels=N_MELS, duration=DURATION):
    y, _ = librosa.load(file_path, sr=sr)
    target_len = int(sr*duration)
    
    if len(y) > target_len:
        y = y[:target_len]
    else:
        y = np.pad(y, (0, target_len - len(y)))

    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    logmel = librosa.power_to_db(mel)
    logmel = (logmel - logmel.mean()) / (logmel.std() + 1e-8)

    return logmel.astype(np.float32)

# Embedding Extraction for a Single File

def embed_file(model, file_path, device='cpu'):
    m = extract_logmel(file_path)
    tensor = torch.tensor(m).unsqueeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        emb, _ = model(tensor)
        emb = emb.cpu().numpy()[0]
    return emb

# Enrollemnt Embedding (Average of N enrollment samples)
def enroll_embedding(model, file_list, device='cpu'):
    embs = [embed_file(model, f, device=device) for f in file_list]
    
    mean = np.stack(embs).mean(axis=0)
    
    return mean / np.linalg.norm(mean)

In [6]:
# Device Configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initalize CNN embedding model
NUM_SPEAKERS = 27
MODEL_PATH = "/kaggle/input/model-cnn/speaker_cnn_trained (1).pth"

model = SpeakerCNN(num_speakers=NUM_SPEAKERS).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Compute enrollment embedding (UNCHANGED)
enroll_emb = enroll_embedding(model, g_enroll, device=device)
print("Enrollment embedding computed")

Enrollment embedding computed


In [7]:
# -----------------------------
# 1. Compute similarity scores
# -----------------------------
pos_dev_sims = np.array([
    float(np.dot(enroll_emb, embed_file(model, f, device=device)))
    for f in g_dev
])
neg_dev_sims = np.array([
    float(np.dot(enroll_emb, embed_file(model, f, device=device)))
    for f in imp_dev
])

# Combine labels and scores
scores = np.concatenate([pos_dev_sims, neg_dev_sims])
labels = np.concatenate([np.ones_like(pos_dev_sims), np.zeros_like(neg_dev_sims)])

# -----------------------------
# 2. Manual threshold
# -----------------------------
threshold = 0.91  # Change this manually
print(f"Using manual threshold: {threshold:.3f}")

y_pred = (scores >= threshold).astype(int)

# Confusion matrix and accuracy
cm = confusion_matrix(labels, y_pred)
acc = accuracy_score(labels, y_pred)
tn, fp, fn, tp = cm.ravel()
FAR = fp / (fp + tn)
FRR = fn / (fn + tp)
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print("\nConfusion Matrix:")
print(cm)
print(f"Accuracy: {acc*100:.2f}%")
print(f"FAR: {FAR*100:.2f}%, FRR: {FRR*100:.2f}%")
print(f"TPR: {tpr*100:.2f}%, FPR: {fpr*100:.2f}%")

# -----------------------------
# PLOT 1 - Similarity Histogram
# -----------------------------
plt.figure(figsize=(8,5))
plt.hist(pos_dev_sims, bins=20, alpha=0.6, label='Genuine')
plt.hist(neg_dev_sims, bins=20, alpha=0.6, label='Imposters')
plt.xlabel('Similarity Score')
plt.ylabel('Count')
plt.title('Similarity Score Distribution')
plt.legend()
plt.tight_layout()
plt.savefig('/kaggle/working/similarity_histogram.png')
plt.close()

# -----------------------------
# PLOT 2 - ROC Curve (Manual Threshold Only)
# -----------------------------
fpr_curve, tpr_curve, thresholds_curve = roc_curve(labels, scores)
roc_auc = auc(fpr_curve, tpr_curve)

plt.figure(figsize=(8,5))
plt.plot(fpr_curve, tpr_curve, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0,1], [0,1], 'k--')

# Mark manual threshold on ROC curve
th_idx = np.argmin(np.abs(thresholds_curve - threshold))
plt.scatter(fpr_curve[th_idx], tpr_curve[th_idx],
            color='red',
            label=f'Manual threshold = {threshold:.3f}')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.tight_layout()
plt.savefig('/kaggle/working/roc_curve.png')
plt.close()

# -----------------------------
# PLOT 3 - FAR/FRR vs Threshold (Manual Only)
# -----------------------------
fnr_curve = 1 - tpr_curve  # needed for FRR curve

plt.figure(figsize=(8,5))
plt.plot(thresholds_curve, fpr_curve, label='FAR')
plt.plot(thresholds_curve, fnr_curve, label='FRR')

# Mark manual threshold
plt.axvline(x=threshold, color='red', linestyle='--',
            label=f'Manual threshold={threshold:.3f}')

plt.xlabel('Threshold')
plt.ylabel('Error Rate')
plt.title('FAR and FRR vs Threshold')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('/kaggle/working/far_frr_vs_threshold.png')
plt.close()

# -----------------------------
# PLOT 4 - Confusion Matrix Counts Bar Plot
# -----------------------------
plt.figure(figsize=(6,4))
plt.bar(['TP','FP','TN','FN'], [tp, fp, tn, fn], color=['green','red','green','red'])
plt.title(f'Confusion Matrix Counts at Threshold {threshold:.2f}')
plt.ylabel('Number of samples')
for i, v in enumerate([tp, fp, tn, fn]):
    plt.text(i, v + 0.5, str(v), ha='center', fontsize=12)
plt.tight_layout()
plt.savefig('/kaggle/working/confusion_matrix_counts.png')
plt.close()

# -----------------------------
# PLOT 5 - TPR/FPR Bar Plot at Manual Threshold
# -----------------------------
plt.figure(figsize=(9,4))
plt.bar(['TPR','FPR'], [tpr, fpr], color=['blue','orange'])
plt.title(f'TPR and FPR at Threshold {threshold:.2f}')
plt.ylim(0,1)
for i, v in enumerate([tpr, fpr]):
    plt.text(i, v + 0.02, f'{v*100:.1f}%', ha='center', fontsize=12)
plt.tight_layout()
plt.savefig('/kaggle/working/tpr_fpr.png')
plt.close()

# SAVE THE PLOTS
print("\nAll plots saved in /kaggle/working/:")
print(" - similarity_histogram.png")
print(" - roc_curve.png")
print(" - far_frr_vs_threshold.png")
print(" - confusion_matrix_counts.png")
print(" - tpr_fpr.png")


Using manual threshold: 0.910

Confusion Matrix:
[[1200    0]
 [   0   60]]
Accuracy: 100.00%
FAR: 0.00%, FRR: 0.00%
TPR: 100.00%, FPR: 0.00%

All plots saved in /kaggle/working/:
 - similarity_histogram.png
 - roc_curve.png
 - far_frr_vs_threshold.png
 - confusion_matrix_counts.png
 - tpr_fpr.png


In [8]:
# Compute similarities for test sets
pos_test_sims = np.array([
    float(np.dot(enroll_emb, embed_file(model,f,device=device)))
    for f in g_test
])
neg_test_sims = np.array([
    float(np.dot(enroll_emb, embed_file(model,f,device=device)))
    for f in imp_test
])

# Compute metrics
tp = np.sum(pos_test_sims >= threshold)
fn = np.sum(pos_test_sims < threshold)
tn = np.sum(neg_test_sims < threshold)
fp = np.sum(neg_test_sims >= threshold)

FAR = fp / (fp + tn)
FRR = fn / (fn + tp)
ACC = (tp + tn) / (len(pos_test_sims) + len(neg_test_sims))

print("\n--- Test Set Performance ---")
print(f"Accuracy: {ACC:.3f}")
print(f"FAR     : {FAR:.3f}")
print(f"FRR     : {FRR:.3f}")


--- Test Set Performance ---
Accuracy: 1.000
FAR     : 0.000
FRR     : 0.000


In [9]:
import glob

# Folders containing genuine and fake test audio
test_folders = [
    '/kaggle/input/test-speakers'
]

# Collect all .wav files recursively
new_test_files = []
for folder in test_folders:
    new_test_files.extend(
        glob.glob(os.path.join(folder, '**', '*.wav'), recursive=True)
    )

print(f"Found {len(new_test_files)} test files")

results = []
label_map = {}  # this will auto-update based on model outputs

# Evaluate each test file
for f in sorted(new_test_files):
    sim = float(np.dot(enroll_emb, embed_file(model, f, device=device)))
    result = 'ACCEPT' if sim >= threshold else 'REJECT'

    # AUTO LABEL based on model result
    label = 'GENUINE' if result == 'ACCEPT' else 'FAKE'
    
    # Save to dict
    fname = os.path.basename(f)
    label_map[fname] = label
    
    # Save row for printing later (optional)
    results.append((label, fname, sim, result))

# Print results table
for label, fname, sim, result in results:
    print(f"{label:8s} | {fname:25s} → sim={sim:.3f} → {result}")

Found 8 test files
FAKE     | S_1.wav                   → sim=0.021 → REJECT
FAKE     | S_2.wav                   → sim=0.105 → REJECT
FAKE     | S_3.wav                   → sim=0.681 → REJECT
FAKE     | S_4.wav                   → sim=0.881 → REJECT
GENUINE  | S_5.wav                   → sim=0.922 → ACCEPT
GENUINE  | S_6.wav                   → sim=0.938 → ACCEPT
GENUINE  | S_7.wav                   → sim=0.921 → ACCEPT
GENUINE  | S_8.wav                   → sim=0.918 → ACCEPT
